In [2]:
import json
import requests
import numpy as np
import pandas as pd
import FinanceDataReader as fdr

In [13]:
date = '20211207'

result = requests.post("http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
                        , headers = {'User-Agent': 'Mozilla/5.0'}
                        , params = dict(bld= 'dbms/MDC/STAT/standard/MDCSTAT03501'
                                , searchType=	1
                                , mktId=	'ALL'
                                , trdDd=	date
                                , isuCd=	'KR7005930003'
                        ))
if not result.content:
    raise ValueError('Empty Result!')


In [14]:

# Make DataFrame
df = pd.DataFrame(json.loads(result.content)['output'])

# Rename Columns, Remove unused columns
columns={
    'DVD_YLD':'배당수익률'
    , 'TDD_CLSPRC':'종가'
    , 'ISU_SRT_CD':'종목코드'
    , 'ISU_ABBRV':'종목명'
}
df = df.rename(columns=columns)
df =df[['종목명', '종목코드', '종가', 'PER', 'PBR', '배당수익률']]
df['기준일'] = date

# Preprocess Variables
for col in ['종가', 'PER', 'PBR', '배당수익률']:
    df[col] = df[col].apply(lambda x: x.replace(',', ''))
df = df.replace(to_replace='-', value=np.NaN)
df = df.replace(to_replace='', value=np.NaN)
df = df.astype({
    '종목명':str,
    '종목코드':str,
    '종가':int,
    'PER':float,
    'PBR':float,
    '배당수익률':float
})